# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, Dataset, DatasetDict

In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1+cu117
CUDA version: 11.7
cuDNN version: 8500
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.32.1
Datasets version: 2.14.4


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Wed Nov 22 16:59:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      On | 00000000:01:00.0 Off |                  N/A |
| 28%   50C    P2               81W / 250W|   2367MiB / 11264MiB |     29%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

|   6  NVIDIA GeForce RTX 2080 Ti      On | 00000000:C1:00.0 Off |                  N/A |
| 40%   67C    P2              241W / 250W|   3641MiB / 11264MiB |     46%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   7  NVIDIA GeForce RTX 2080 Ti      On | 00000000:E1:00.0 Off |                  N/A |
| 41%   67C    P2              188W / 250W|  10431MiB / 11264MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   G

|    6   N/A  N/A   2025669      C   python3                                    3636MiB |
|    7   N/A  N/A   1390518      C   ...ml/home/limeng/torch_env/bin/python    10424MiB |
+---------------------------------------------------------------------------------------+


In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU  | MEM |
-------------------
|  0 |  29% | 21% |
|  1 | 100% | 93% |
|  2 | 100% | 93% |
|  3 | 100% | 93% |
|  4 |   0% | 12% |
|  5 |  92% | 32% |
|  6 |  46% | 32% |
|  7 |  99% | 93% |


GPU Usage after emptying the cache
| ID | GPU  | MEM |
-------------------
|  0 |  31% | 22% |
|  1 |  99% | 93% |
|  2 | 100% | 93% |
|  3 | 100% | 93% |
|  4 |   0% | 12% |
|  5 |  92% | 32% |
|  6 |  91% | 32% |
|  7 |  99% | 93% |


In [8]:
data = pd.read_csv("csv/priority_high_low_clean.csv" , index_col = 0)
data

,text_clean,label
0,is your feature request related to a problem p...,0
1,implement an android image picker to browse im...,0
2,when a video is not playing hover should displ...,0
3,description escape shuttle accelerates faster ...,0
4,httpsnotebooksgesisorgbinderjupyteruserrussted...,0
...,...,...
444474,cve202024977 medium severity vulnerability det...,1
444475,tags javascript css php html sql css php html ...,1
444476,we can access a specific tflite model through ...,1
444477,autotuneclientregistermodels should be autotun...,1


In [9]:
# Smaller and faster than bert.
base_model_id = "distilbert-base-uncased"

epochs = 5 #Number of full cyles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evalutaion examples in on iteratoion.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

In [10]:

# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [11]:
# Drops rows with missing values
data.dropna(inplace=True)

In [12]:
# Resets the index after dropping rows
data.reset_index(inplace=True)
data

,index,text_clean,label
0,0,is your feature request related to a problem p...,0
1,1,implement an android image picker to browse im...,0
2,2,when a video is not playing hover should displ...,0
3,3,description escape shuttle accelerates faster ...,0
4,4,httpsnotebooksgesisorgbinderjupyteruserrussted...,0
...,...,...,...
444474,444474,cve202024977 medium severity vulnerability det...,1
444475,444475,tags javascript css php html sql css php html ...,1
444476,444476,we can access a specific tflite model through ...,1
444477,444477,autotuneclientregistermodels should be autotun...,1


In [13]:
#Drops the index col, better for managint the data.
data.drop(columns= ["index"], inplace = True)

In [14]:
data

,text_clean,label
0,is your feature request related to a problem p...,0
1,implement an android image picker to browse im...,0
2,when a video is not playing hover should displ...,0
3,description escape shuttle accelerates faster ...,0
4,httpsnotebooksgesisorgbinderjupyteruserrussted...,0
...,...,...
444474,cve202024977 medium severity vulnerability det...,1
444475,tags javascript css php html sql css php html ...,1
444476,we can access a specific tflite model through ...,1
444477,autotuneclientregistermodels should be autotun...,1


In [15]:
# 60% trainig, 20% validate, 20% test. Seed None.
# Test 80-10-10 and 70-15-15
train , validate , test = train_validate_test_split(data)

In [16]:
train.set_index("label" , inplace = True)
validate.set_index("label" , inplace = True)
test.set_index("label" , inplace = True)

In [17]:
test

,text_clean
label,
1,feature request is your feature request relate...
1,thank you for filing a bug report please prov...
0,version download master branche at 0100 pm tod...
0,today in the getting started for serving we do...
1,it is a good product appreceiate for developme...
...,...
1,might just be filling out the config might inv...
1,there are 4 different notificationscontrollers...
0,found via 1342 want to make sure the saved tex...


In [18]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
testds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = testds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 44449
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 355583
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 44447
    })
})

In [19]:
train_dataset = ds["train"]
valid_dataset = ds["validate"]
test_ds = ds["test"]

In [20]:
ds["train"][0]

{'text_clean': 'as a relationship manager i want outbound calls to be automatically dialed by the system so that i can focus on sales and not dailing numbers from a list',
 'label': 0}

In [21]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=num_labels)
#tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenization

In [23]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [24]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=len(valid_dataset))
test_dataset = test_ds.map(tokenize, batched=True, batch_size=len(test_ds))

Map:   0%|          | 0/355583 [00:00<?, ? examples/s]

Map:   0%|          | 0/44447 [00:00<?, ? examples/s]

Map:   0%|          | 0/44449 [00:00<?, ? examples/s]

## Training a classifier

In [25]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [26]:
 trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

In [27]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, valdiation loss increases = Overfitting

In [ ]:
# Evaluate valdiation set
eval_result = trainer.evaluate(eval_dataset=valid_dataset)

In [ ]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

In [ ]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

In [ ]:
trainer.save_model(model_dir + "_local") 

In [ ]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [ ]:
classifier.model

In [ ]:
classifier("this does not need to be done fast")

In [ ]:
classifier("this is super important")

In [ ]:
classifier("this bug has super high impact on the project")

## Important to delete large objects to free memory 
del train_dataset

In [ ]:
del valid_dataset

In [ ]:
del model

In [ ]:
# Free cache
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi